Illustration of built-in dataset using tensorflow_datasets module

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
# Can use below simple code to download 'mnist' dataset
#datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)
#mnist_train, mnist_test = datasets['train'], datasets['test']

Below function loads mnist data using tensorflow_datasets, split to train, validation and test.

In [3]:
def load_mnist():
    """ load tensorflow mnist builtin dataset """
    # Split the training data to 90, and 10 %
    ds_train_s, ds_validate_s = tfds.Split.TRAIN.subsplit([9, 1])
    # Download and load three datasets directly
    tfds_train, tfds_validate, tfds_test = \
        tfds.load(name='mnist',
            split=[ds_train_s, ds_validate_s, tfds.Split.TEST], as_supervised=True)
    return tfds_train, tfds_validate, tfds_test

In [4]:
mnist_train, mnist_validate, mnist_test = load_mnist()

In [5]:
BUFFER_SIZE = 10 # Use a much larger value for real code.
BATCH_SIZE = 64
NUM_EPOCHS = 5

In [6]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

In [7]:
train_data = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE).take(5)
validation_data = mnist_validate.map(scale).batch(BATCH_SIZE).take(5)
test_data = mnist_test.map(scale).batch(BATCH_SIZE).take(5)

STEPS_PER_EPOCH = 5

train_data = train_data.take(STEPS_PER_EPOCH)
validation_data = validation_data.take(STEPS_PER_EPOCH)
test_data = test_data.take(STEPS_PER_EPOCH)

In [8]:
image_batch, label_batch = next(iter(train_data))
print(image_batch.shape)

(64, 28, 28, 1)


In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.02),
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Model is the full model w/o custom layers
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(train_data, validation_data=validation_data, epochs=NUM_EPOCHS)

Epoch 1/5
5/5 [==============================] - 2s 315ms/step - loss: 1.4859 - accuracy: 0.5281 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
5/5 [==============================] - 1s 104ms/step - loss: 0.5547 - accuracy: 0.8687 - val_loss: 1.8774 - val_accuracy: 0.5063
Epoch 3/5
5/5 [==============================] - 1s 160ms/step - loss: 0.3758 - accuracy: 0.9375 - val_loss: 1.7765 - val_accuracy: 0.5656
Epoch 4/5
5/5 [==============================] - 0s 95ms/step - loss: 0.2830 - accuracy: 0.9625 - val_loss: 1.7061 - val_accuracy: 0.6281
Epoch 5/5
5/5 [==============================] - 1s 107ms/step - loss: 0.2097 - accuracy: 0.9875 - val_loss: 1.6508 - val_accuracy: 0.6656


In [11]:
loss, acc = model.evaluate(test_data)
print("Loss {}, Accuracy {}".format(loss, acc))

      5/Unknown - 0s 26ms/step - loss: 1.6211 - accuracy: 0.6687Loss 1.6210591554641725, Accuracy 0.668749988079071
